# Feature Selection and LASSO

In [1]:
import turicreate as tc

In [3]:
sales = tc.SFrame('home_data.SFrame')

In [4]:
from math import log, sqrt

sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.

sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

# Learn regression weights with L1 penalty

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [6]:
model_all = tc.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.610515     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.993701     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [8]:
# non_zero_weights

non_zero_weight = model_all.coefficients[model_all.coefficients["value"] > 0]
non_zero_weight.print_rows(num_rows=20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.0559504957  |  None  |
|    bathrooms     |  None | 8468.531086910072  |  None  |
|   sqft_living    |  None | 24.420720982445214 |  None  |
| sqft_living_sqrt |  None | 350.0605533860648  |  None  |
|      grade       |  None | 842.0680348976282  |  None  |
|    sqft_above    |  None | 20.024722417091304 |  None  |
+------------------+-------+--------------------+--------+
[6 rows x 4 columns]



# Selecting an L1 penalty

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [10]:
import numpy as np


validation_rss = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss

# pprint.pprint(result_dict)
print (min(validation_rss.items(), key=lambda x: x[1]))

(10.0, 625766285142461.2)


In [12]:
model_test = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10.0)
predictions_test = model.predict(testing)
residuals_test = testing['price'] - predictions_test
rss_test = sum(residuals_test**2)
print (rss_test)

#1.56972779669e+14

156972779668688.7


In [13]:
non_zero_weight_test = model_test.coefficients[model_test.coefficients["value"] > 0]

print (model_test.coefficients["value"].nnz())

non_zero_weight_test.print_rows(num_rows=20)

18
+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  18993.427212770577  |  None  |
|     bedrooms     |  None |  7936.967679031306   |  None  |
| bedrooms_square  |  None |   936.993368193299   |  None  |
|    bathrooms     |  None |  25409.58893412067   |  None  |
|   sqft_living    |  None |  39.11513637970762   |  None  |
| sqft_living_sqrt |  None |  1124.6502128077207  |  None  |
|     sqft_lot     |  None | 0.003483618222989743 |  None  |
|  sqft_lot_sqrt   |  None |  148.2583910114082   |  None  |
|      floors      |  None |  21204.33546695013   |  None  |
|  floors_square   |  None |  12915.524336072436  |  None  |
|    waterfront    |  None |  601905.5945452718   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       | 

# Limit the number of nonzero weights

In [14]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity


In [15]:
l1_penalty_values = np.logspace(8, 10, num=20)
print (l1_penalty_values)

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]


In [16]:
coef_dict = {}
for l1_penalty in l1_penalty_values:
    model = tc.linear_regression.create(training, target ='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    coef_dict[l1_penalty] = model.coefficients['value'].nnz()

pprint.pprint(coef_dict)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}


In [17]:
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set

In [18]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [21]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss, model.coefficients['value'].nnz()


In [72]:

for k,v in validation_rss.items():    
    if (v[1] == max_nonzeros) and (v[0] < rss):
        bestRSS = v[0]
        bestl1 = k
        
print(bestRSS, bestl1)

1060799531763290.4 3577864204.126743


In [74]:
for k,v in validation_rss.items():
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        print (k, bestRSS)

3448968612.1634364 1046937488751713.6


In [75]:
model = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3448968612.16)

In [76]:
non_zero_weight_test = model.coefficients[model.coefficients["value"] > 0]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.19254432785 |  None  |
|     bedrooms     |  None | 661.7227177822587  |  None  |
|    bathrooms     |  None | 15873.957259267981 |  None  |
|   sqft_living    |  None | 32.41022145125964  |  None  |
| sqft_living_sqrt |  None | 690.1147733133256  |  None  |
|      grade       |  None | 2899.4202697498786 |  None  |
|    sqft_above    |  None | 30.011575302201045 |  None  |
+------------------+-------+--------------------+--------+
[7 rows x 4 columns]



In [78]:
sales = tc.SFrame('home_data.SFrame')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below

sales['floors'] = sales['floors'].astype(int)

In [79]:
def get_numpy_data(data_sframe, features, output):
    
    data_sframe['constant'] = 1 
    
    # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    
    features = ['constant'] + features
    
    # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    
    features_sframe = data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    
    output_sarray = data_sframe[output]
    
    # the following will convert the SArray into a numpy array by first converting it to a list
    
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [80]:
def predict_output(feature_matrix, weights):
    
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

In [81]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [83]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return (normalized_features, norms)


In [84]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))

In [85]:
print(features,norms)

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]] [ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

In [86]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [87]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [88]:
weights = np.array([1., 4., 1.])

In [89]:
prediction = predict_output(simple_feature_matrix, weights)

In [90]:
ro = [0 for i in range((simple_feature_matrix.shape)[1])]
for j in range((simple_feature_matrix.shape)[1]):   
    ro[j] = (simple_feature_matrix[:,j] * (output - prediction + (weights[j] * simple_feature_matrix[:,j]))).sum()
print (ro)

[79400300.03492916, 87939470.77299108, 80966698.67596565]


In [92]:
print (2* ro[1])
print (2* ro[2])

175878941.54598215
161933397.3519313


In [93]:
# Looking for range -l1_penalty/2 <= ro <= l1_penalty/2
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value <= penalty/2.) )

In [95]:
for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print (in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty))

False False
False True
False True
True True
True True


# Single Coordinate Descent Step

In [96]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

#  Cyclical coordinate descent

In [97]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False

    while not converged:

    # Evaluate over all features
        for idx in range(D):
#             print 'Feature: ' + str(idx)
            # new weight for feature
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            # compute change in weight for feature
            change[idx] = np.abs(new_weight - weights[idx])
#             print '  -> old weight: ' + str(weights[idx]) + ', new weight: ' + str(new_weight)
#             print '  -> abs change (new - old): ' + str(change[idx])
#             print '  >> old weights: ', weights

            # assign new weight
            weights[idx] = new_weight
#             print '  >> new weights: ', weights
        # maximum change in weight, after all changes have been computed
        max_change = max(change)
#         print '  ** max change: ' + str(max_change)
#         print '--------------------------------------------------'
        if max_change < tolerance:
            converged = True
    return weights

In [98]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [99]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [100]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [101]:
print(weights)

[21624998.36636292 63157246.78545421        0.        ]


In [102]:
prediction =  predict_output(normalized_simple_feature_matrix, weights)
RSS = np.dot(output-prediction, output-prediction)
print ('RSS for normalized dataset = ', RSS)

RSS for normalized dataset =  1630492481484487.5


# Evaluating LASSO fit with more features

In [103]:
train_data,test_data = sales.random_split(.8,seed=0)

In [104]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [105]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [106]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0

In [107]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

print (weights1e7)


[24429600.60933314        0.                0.         48389174.35227978
        0.                0.          3317511.16271981  7329961.9848964
        0.                0.                0.                0.
        0.                0.        ]


In [108]:
feature_list = ['constant'] + all_features
print (feature_list)

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [109]:
feature_weights1e7 = dict(zip(feature_list, weights1e7))
for k,v in feature_weights1e7.items():
    if v != 0.0:
        print (k, v)

constant 24429600.609333135
sqft_living 48389174.35227978
waterfront 3317511.162719814
view 7329961.984896399


In [110]:

l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [111]:
print (weights1e8)

[71114625.75280938        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


In [113]:
feature_weights1e8 = dict(zip(feature_list, weights1e8))
for k,v in feature_weights1e8.items():
    if v != 0.0:
        print (k, v)

constant 71114625.75280938


In [114]:
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [115]:
print (weights1e4)

[ 77779073.91265218 -22884012.25023358  15348487.08089995
  92166869.6988308   -2139328.0824278   -8818455.54409494
   6494209.73310655   7065162.05053197   4119079.21006765
  18436483.5261878  -14566678.54514346  -5528348.75179428
 -83591746.20730534   2784276.46012858]


In [116]:

feature_weights1e4 = dict(zip(feature_list, weights1e4))
for k,v in feature_weights1e4.items():
    if v != 0.0:
        print (k, v)

constant 77779073.91265218
bedrooms -22884012.250233576
bathrooms 15348487.080899946
sqft_living 92166869.6988308
sqft_lot -2139328.0824277992
floors -8818455.544094937
waterfront 6494209.733106552
view 7065162.050531974
condition 4119079.210067647
grade 18436483.5261878
sqft_above -14566678.545143465
sqft_basement -5528348.751794276
yr_built -83591746.20730534
yr_renovated 2784276.460128578


In [117]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print (normalized_weights1e7[3])

161.3174562483786


In [118]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [119]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
RSS = np.dot(test_output-prediction, test_output-prediction)
print ('RSS for model with weights1e7 = ', RSS)

RSS for model with weights1e7 =  275962079909185.25


In [120]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
RSS = np.dot(test_output-prediction, test_output-prediction)
print ('RSS for model with weights1e8 = ', RSS)

RSS for model with weights1e8 =  537166150034084.94


In [121]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
RSS = np.dot(test_output-prediction, test_output-prediction)
print ('RSS for model with weights1e4 = ', RSS)

RSS for model with weights1e4 =  227781004760225.28
